***Library Imports***

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import keras.backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

***Connect Google Drive***

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


***DATA LOADING AND PROCESSING***

In [ ]:
data_dir='/content/drive/MyDrive/covid_multiclass_classification/data'
train_datagen = ImageDataGenerator(rescale = 1./255, 
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True)

valid_datagen = ImageDataGenerator(rescale = 1./255)

***Training and Validation Set***

In [ ]:
input_size = (128,128)
training_set = train_datagen.flow_from_directory("/content/drive/MyDrive/covid_multiclass_classification/data/train",  
                                                 target_size = input_size, 
                                                 color_mode="rgb",
                                                 batch_size = 16, 
                                                 class_mode = 'categorical')

validation_set = valid_datagen.flow_from_directory(
                                  "/content/drive/MyDrive/covid_multiclass_classification/data/val", 
                                  target_size=input_size,
                                  color_mode="rgb",
                                  shuffle = False,
                                  class_mode='categorical',
                                  batch_size=1)

test_set = valid_datagen.flow_from_directory(
                                  "/content/drive/MyDrive/covid_multiclass_classification/data/test", 
                                  target_size=input_size,
                                  color_mode="rgb",
                                  shuffle = False,
                                  class_mode='categorical',
                                  batch_size=1)

Found 3454 images belonging to 3 classes.
Found 72 images belonging to 3 classes.
Found 360 images belonging to 3 classes.




***Baseline Model Architechture***

Inception Modeule

In [3]:
def inception_module(layer_in, f1, f2, f3):
	# 1x1 conv
	conv1 = Conv2D(f1, (1,1), padding='same', activation='selu')(layer_in)
	# 3x3 conv
	conv3 = Conv2D(f2, (3,3), padding='same', activation='selu')(layer_in)
	# 5x5 conv
	conv5 = Conv2D(f3, (5,5), padding='same', activation='selu')(layer_in)
	# concatenate filters, assumes filters/channels last
	layer_out = concatenate([conv1, conv3, conv5], axis=-1)
	return layer_out

Model Definition

In [4]:
def InceptionVgg(input_size= (128,128,3),classes=3):

    img_input = Input(input_size)

    #Inception Block
    inception= inception_module(img_input, 16,16,32)

    # Block 1
    x = Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='selu', name='block1_conv1')(inception)

    x = Conv2D(filters=64, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='selu', name='block1_conv2')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block1_pool", padding='valid')(x)

    # Block 2
    x = Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='selu', name='block2_conv1')(x)

    x = Conv2D(filters=128, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='selu', name='block2_conv2')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block2_pool", padding='valid')(x)

    # Block 3
    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='selu', name='block3_conv1')(x)

    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='selu', name='block3_conv2')(x)

    x = Conv2D(filters=256, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='selu', name='block3_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block3_pool", padding='valid')(x)

    # Block 4
    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='selu', name='block4_conv1')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='selu', name='block4_conv2')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='selu', name='block4_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block4_pool", padding='valid')(x)

    # Block 5
    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='selu', name='block5_conv1')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='selu', name='block5_conv2')(x)

    x = Conv2D(filters=512, kernel_size=3, strides=(1, 1), padding='same',
               kernel_regularizer=l2(0.0002),
               activation='selu', name='block5_conv3')(x)

    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name="block5_pool", padding='valid')(x)

   
    # Classification block
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='selu', name='fc1')(x)
    x = Dropout(0.5, name='drop_fc1')(x)

    x = Dense(2048, activation='selu', name='fc2')(x)
    x = Dropout(0.5, name='drop_fc2')(x)
    
    x = Dense(classes, activation='softmax', name="predictions")(x)


    # Create model.
    model = Model(img_input, x, name='vgg16')

    return model
model= InceptionVgg()

loss= [tf.keras.losses.categorical_crossentropy, tf.keras.losses.kl_divergence]

model.compile(optimizer = Adam(learning_rate = 0.0001), 
                       loss= loss,metrics = ['categorical_accuracy'])
#proposed_model.summary()

***Base line Model Training***

In [ ]:
LR_callback= [tf.keras.callbacks.ReduceLROnPlateau(patience= 5, verbose= 1, monitor= 'loss')]


model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/covid_multiclass_classification/best_model.h5',
                                                               save_weights_only=True,val_loss='loss',
                                                               mode='min',save_best_only=True,verbose=2)

history= model.fit( training_set,validation_data= validation_set, callbacks=[LR_callback, model_checkpoint_callback],
                   epochs=50,shuffle= True)

Epoch 1/50
215/215 [==============================] - 88s 383ms/step - loss: 6.3668 - categorical_accuracy: 0.3959 - val_loss: 2.3352 - val_categorical_accuracy: 0.5972

Epoch 00001: val_loss improved from inf to 2.33519, saving model to /content/drive/MyDrive/covid_multiclass_classification/best_model.h5
Epoch 2/50
215/215 [==============================] - 80s 370ms/step - loss: 4.2387 - categorical_accuracy: 0.5154 - val_loss: 2.2225 - val_categorical_accuracy: 0.6944

Epoch 00002: val_loss improved from 2.33519 to 2.22254, saving model to /content/drive/MyDrive/covid_multiclass_classification/best_model.h5
Epoch 3/50
215/215 [==============================] - 78s 360ms/step - loss: 3.0296 - categorical_accuracy: 0.6111 - val_loss: 1.6517 - val_categorical_accuracy: 0.6944

Epoch 00003: val_loss improved from 2.22254 to 1.65171, saving model to /content/drive/MyDrive/covid_multiclass_classification/best_model.h5
Epoch 4/50
215/215 [==============================] - 78s 361ms/step - 

***Load weights for best model***

In [ ]:
model.load_weights('/content/drive/MyDrive/covid_multiclass_classification/best_model.h5')

***Prediction on Test Set***

In [ ]:
filenames = test_set.filenames
nb_samples = len(filenames)
y_test= test_set.classes

In [ ]:
import numpy as np
test = model.predict(test_set,steps = nb_samples, verbose=1)
pred= test.round()
preds = (np.argmax(pred==1, axis=1))

360/360 [==============================] - 8s 22ms/step


In [ ]:
accuracy = accuracy_score(y_test, preds)
print('Accuracy of the model on {} test images:'.format(len(preds)), accuracy)

Accuracy of the model on 360 test images: 0.9527777777777777


***Confusion Matix and Weighted Accuracy***

In [ ]:
#Confusion Matrix and Weighted accuracy
cm=confusion_matrix(y_test,preds, normalize= None)
class_accuracy_score= np.diag(cm)
sample_in_each_class= np.sum(cm, axis=1)
class_acc= np.true_divide(class_accuracy_score, sample_in_each_class)
Weighted_accuracy= np.sum(np.multiply(class_acc, sample_in_each_class))/(np.sum(sample_in_each_class))
print(Weighted_accuracy)

0.9527777777777777


In [ ]:
np.save('/content/drive/MyDrive/covid_multiclass_classification/y_test.np', y_test)
np.save('/content/drive/MyDrive/covid_multiclass_classification/preds.np', preds)